Fluff to get the relative imports working in a notebook

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path

'C:\\Users\\micro_zo50ceu\\OneDrive - University College London\\BUCLSE'

# Timer

In [2]:
from UCLSE.custom_timer import CustomTimer

All of the objects we are about to explore need to agree about the current time. We pass them a simple timer. In the first instance this stops the need of passing a time variable around between objects. 

This is how we instantiate it:

In [3]:
timer=CustomTimer(start=0,end=10,step=1)

In [4]:
timer.duration

10.0

In [5]:
timer.get_time, timer.get_time_left

(0, 10.0)

In [6]:
print(timer)

time: 0 time left: 10.0 start: 0 end: 10 step: 1


To increment the time, we can call the following (it will only work if there is time remaining)

In [7]:
timer.next_period()
timer

time: 1 time left: 9.0 start: 0 end: 10 step: 1

# Messenger

All of the objects we are about to explore typically communicate with each other using a messenger object. This is how we instantiate it. The logging=True flag makes sure that messages are saved. Careful, we might not always want to save messages as this is a significant overhead.

In [8]:
from UCLSE.messenger import Messenger,Message

messenger=Messenger(logging=True)

# Exchange

In [9]:
from UCLSE.message_exchange import Exchange

An exchange object can be instantiated as follows:

In [10]:
exchange=Exchange(timer=timer,messenger=messenger,record=True)

In [11]:
print(exchange)

No orders in exchange order book


In [12]:
exchange.__dict__

{'bids': <UCLSE.exchange.Orderbook_half at 0x17529aa0d08>,
 'asks': <UCLSE.exchange.Orderbook_half at 0x175294ad408>,
 'tape': [],
 'quote_id': 0,
 'timer': time: 1 time left: 9.0 start: 0 end: 10 step: 1,
 'name': 'exchange1',
 'record': True,
 'lob_call': deque([(0, 0)]),
 'messenger': logging: True, subscribed: 1}

The exchange is composed of two 'Orderbook_halfs'; each corresponding to ordered lists of bids and asks. 

Tape refers to the trades that have gone through the exchange to date. 

Quote_id is an internal counter which assigns a unique quote_id to each order placed on exchange

Timer is an object that when queried, tells the exchange what time it is. More on that later.

Lob call records when the last anonymised lob was constructed and gives it an ascending version number (to track different versions within the same time period)

# Trader

In [13]:
from UCLSE.message_trader import TraderM

A trader object can be instantiated as follows:

In [14]:
henry=TraderM(tid='Henry',exchange=exchange,timer=timer, messenger=messenger)

The particulars of the trader can be viewed by:

In [15]:
print(henry)

[TID: Henry type: None balance: 0 blotter: {} orders: OrderedDict() n_trades: 0 profitpertime: 0]


Or for a more in depth look:

In [16]:
henry.__dict__

{'ttype': None,
 'tid': 'Henry',
 'balance': 0,
 'blotter': {},
 'orders': [],
 'orders_dic': OrderedDict(),
 'history': False,
 'orders_lookup': {},
 'n_orders': 0,
 'n_quotes': 0,
 'n_quote_limit': 1,
 'profitpertime': 0,
 'n_trades': 0,
 'lastquote': {},
 'latency': 1,
 'total_quotes': 0,
 'timer': time: 1 time left: 9.0 start: 0 end: 10 step: 1,
 'birthtime': 1,
 'exchange': No orders in exchange order book,
 'last_quote_time': 1,
 'name': 'Henry',
 'messenger': logging: True, subscribed: 2}

The trader object is the parent class for multiple objects which represent specific types of trader. Included are Giveaway, ZIC (After Gode & Sunder 1993), Shaver, Sniper and ZIP (After Cliff 1997). 

Trader objects are differentiated by their getorder and respond methods. The former will produce an order when called, the second will update internal variables when called. 

If traders want to trade then they need to send orders to an exchange....

# Orders

In [17]:
from UCLSE.exchange import Order

Traders submit orders to an exchange. An order is a namedtuple object which contains trader id, order type (Bid or Ask), price, quantity, time, quote_id and order_id.

In [18]:
order=Order('henry',otype='Bid',qty=1,price=100,time=1,oid=1)

Named tuples are cool:

In [19]:
print(order)

Order(tid='henry', otype='Bid', price=100, qty=1, time=1, qid=None, oid=1)


In [20]:
order.tid

'henry'

And they are mostly immutable, like normal tuples:

In [21]:
order.tid='John'

AttributeError: can't set attribute

# Messages and the Messenger (again)

Let's subscribe ourselves to the exchange, and send a message to henry containing the order. 

In [22]:
messenger.subscribe(name='a god',tipe='deity',obj=None)

To verify, we can look at the directory of subscribers:

In [23]:
messenger.directory

{'exchange1': DirectoryEntry(name='exchange1', type='Exchange', object=No orders in exchange order book),
 'Henry': DirectoryEntry(name='Henry', type='Trader', object=[TID: Henry type: None balance: 0 blotter: {} orders: OrderedDict() n_trades: 0 profitpertime: 0]),
 'a god': DirectoryEntry(name='a god', type='deity', object=None)}

We can send a message to the messenger which sends it onward.

A message object is another example of the named tuple. 

In [24]:
Message._fields

('too', 'fromm', 'subject', 'time', 'order')

Let's add the order to henry (from the perspective of a client giving him the order to execute)

In [25]:
message=Message(too='Henry',fromm='a god',subject='New Customer Order',order=order,time=timer.time)
messenger.send(message)

And check that henry received it

In [26]:
henry.__dict__

{'ttype': None,
 'tid': 'Henry',
 'balance': 0,
 'blotter': {},
 'orders': [],
 'orders_dic': OrderedDict([(1,
               {'Original': Order(tid='henry', otype='Bid', price=100, qty=1, time=1, qid=None, oid=1),
                'submitted_quotes': [],
                'qty_remain': 1})]),
 'history': False,
 'orders_lookup': {},
 'n_orders': 1,
 'n_quotes': 0,
 'n_quote_limit': 1,
 'profitpertime': 0,
 'n_trades': 0,
 'lastquote': {},
 'latency': 1,
 'total_quotes': 0,
 'timer': time: 1 time left: 9.0 start: 0 end: 10 step: 1,
 'birthtime': 1,
 'exchange': No orders in exchange order book,
 'last_quote_time': 1,
 'name': 'Henry',
 'messenger': logging: True, subscribed: 3}

The internal variables indicate that he has one order, and that it hasn't been submitted to the exchange (n_quotes=0)

Let's send an order to the exchange. traders do so when they receive a 'Get Order' message.

In [27]:
message=Message(too='Henry',fromm='a god',subject='Get Order',time=timer.time,order=None)
messenger.send(message)

Message(too='Henry', fromm='a god', subject='Get Order', time=1, order=None)


AttributeError: 'TraderM' object has no attribute 'getorder'

oops. The raw TraderM object has no method ('getorder') to create orders from its internal list of client orders. This is one of the ways that trader classes are differentiated from each other. Let's quickly write one:





In [28]:
class giveaway(TraderM):
   def getorder(self,a,b,c):
        submit_order_dic={}
        for oid,o in self.orders_dic.items():
            order=Order('henry',otype='Bid',qty=1,price=100,time=1,oid=1)
            cust_order=o['Original']
            new_order=Order(self.tid,otype=cust_order.otype,qty=cust_order.qty,
                            price=cust_order.price,time=self.time,oid=cust_order.oid)
            submit_order_dic[oid]=new_order
            
        return submit_order_dic

This get order method just passes through client orders verbatim to the exchange, making no price improvement.

In [29]:
timer.next_period()    
henry=giveaway(tid='Henry',exchange=exchange,timer=timer, messenger=messenger)


message=Message(too='Henry',fromm='a god',subject='New Customer Order',order=order,time=timer.time)
messenger.send(message)

message=Message(too='Henry',fromm='a god',subject='Get Order',time=timer.time,order=None)
messenger.send(message)

C:\Users\micro_zo50ceu\OneDrive - University College London\BUCLSE\UCLSE\messenger.py:29: UserWarning: Henry is already subscribed, replacing position in directory
  warnings.warn(m,UserWarning)


To check this has arrived at the exchange:

In [30]:
exchange

                          tid
otype                     Bid
price time qid oid qty       
100   2    0   1   1    Henry

Let's look at what happened by examining the messenger log. 
1. god sends customer order to Henry
2. god prompts Henry to submit an order to exchange
3. Henry sends message to exchange containing order
4. Exchange replies with the quote id for the new trade on the exchange.

In [31]:
messenger.publish_log(timer.get_time)[['fromm','too','subject','time','order']]

,fromm,too,subject,time,order
0,a god,Henry,New Customer Order,2,"(henry, Bid, 100, 1, 1, None, 1)"
1,a god,Henry,Get Order,2,None
2,Henry,exchange1,New Exchange Order,2,"(Henry, Bid, 100, 1, 2, None, 1)"
3,exchange1,Henry,Confirm,2,"(Henry, Bid, 100, 1, 2, 0, 1)"


You can see this record at the exchange either by the lob variable which is a dictionary of Order queues (called orderlist) indexed by price. 

The orderlist is a modified deque which holds the orders.

Deque - a datatype that efficiently allows us to append to the back and subtract from the front.

In [32]:
exchange.bids.lob

{100: OrderList([Order(tid='Henry', otype='Bid', price=100, qty=1, time=2, qid=0, oid=1)])}

or in the dictionary of all orders, indexed by qid:

In [33]:
exchange.bids.q_orders

{0: Order(tid='Henry', otype='Bid', price=100, qty=1, time=2, qid=0, oid=1)}

Traders store orders in an ordered dictionary 'orders_dic', the key is OID, which is unique per order.

Within each entry there is a dictionary containing:
- 'Original' orders which do not change
- A list submitted of quotes at the exchange which they will probably alter over time.
- A record of how much quantity of the original order is yet to be executed.

In [35]:
henry.orders_dic

OrderedDict([(1,
              {'Original': Order(tid='henry', otype='Bid', price=100, qty=1, time=1, qid=None, oid=1),
               'submitted_quotes': [Order(tid='Henry', otype='Bid', price=100, qty=1, time=2, qid=0, oid=1)],
               'qty_remain': 1})])

Let's illustrate how a trade gets executed. 

In [36]:
from UCLSE.test.utils import (yamlLoad, order_from_dic,build_df_from_dic_dic,build_lob_from_df,
                               lob_to_dic)
import pandas as pd
import os

First from a fixtures file, we can create a dataframe containing order information

In [37]:
cwd=os.getcwd()
fixture_name=os.path.join(os.path.dirname(cwd),'UCLSE','test','fixtures','exchange_fix.yml')
fixture_list=yamlLoad(fixture_name)
fixture_dic=fixture_list[0]

order_df=build_df_from_dic_dic(fixture_dic['input'])
order_df.set_index('index')

,otype,price,qid,qty,tid,time,oid
index,,,,,,,
0,Ask,107,1000,1,0,0,0
0,Bid,100,1000,1,0,0,1
1,Ask,108,1001,1,1,1,2
1,Bid,101,1001,1,1,1,3
2,Ask,109,1002,1,2,2,4
2,Bid,102,1002,1,2,2,5
3,Ask,110,1003,1,3,3,6
3,Bid,103,1003,1,3,3,7
4,Ask,111,1004,1,4,4,8


This function is in the utilities file but recreated here, it iterates through rows of DF, converts them to an order object then adds to exchange.

In [38]:
def build_lob_from_df(order_df,exch=None,necessary_cols=['tid','otype','price','qty','qid','oid']):
    ##adds orders from a df of orders, if supplied an exchange, will append them
    #else will create blank exchange
    #returns an exchange
    order_df=order_df[necessary_cols]

    if exch is None:
        exch=Exchange(timer=CustomTimer(),messenger=messenger)

    order_list=[]
    for index, row in order_df.iterrows():
        o=row.to_dict()
        #o.pop('index')
        exch.process_new_order(Order(time=exch.time,**o),verbose=False)

    return exch

In [59]:
from UCLSE.custom_timer import CustomTimer
from UCLSE.message_exchange import Exchange
from UCLSE.exchange import  Order
from UCLSE.messenger import Messenger,Message
from UCLSE.message_trader import TraderM

timer=CustomTimer()
messenger=Messenger(logging=True)

exchange=Exchange(timer=timer,messenger=messenger)

messenger.set_asserting(False)
exchange=build_lob_from_df(order_df,exch=exchange)

Recipient not subscribed: 0
Recipient not subscribed: 0
Recipient not subscribed: 1
Recipient not subscribed: 1
Recipient not subscribed: 2
Recipient not subscribed: 2
Recipient not subscribed: 3
Recipient not subscribed: 3
Recipient not subscribed: 4
Recipient not subscribed: 4
Recipient not subscribed: 5
Recipient not subscribed: 5


And we can see that what the LOB looks like:

In [60]:
print(exchange)

                        tid     
otype                   Ask  Bid
price time qid oid qty          
100   0    1   1   1    NaN  0.0
101   0    3   3   1    NaN  1.0
102   0    5   5   1    NaN  2.0
103   0    7   7   1    NaN  3.0
104   0    9   9   1    NaN  4.0
105   0    11  11  1    NaN  5.0
107   0    0   0   1    0.0  NaN
108   0    2   2   1    1.0  NaN
109   0    4   4   1    2.0  NaN
110   0    6   6   1    3.0  NaN
111   0    8   8   1    4.0  NaN
112   0    10  10  1    5.0  NaN


Next period the trader receives the following order:

In [61]:
timer.next_period()

henry=giveaway(tid='Henry',exchange=exchange,timer=timer,time=0,messenger=messenger,history=True) 
#time input manually specifies trader birthtime
new_order=Order(tid='Henry', otype='Bid', price=109, qty=5, qid=None,time=henry.time, oid=50)

message=Message(too='Henry',fromm='a god',subject='New Customer Order',order=new_order,time=timer.time)
messenger.send(message)

message=Message(too='Henry',fromm='a god',subject='Get Order',time=timer.time,order=None)
messenger.send(message)

Recipient not subscribed: 0
Recipient not subscribed: 1
Recipient not subscribed: 2


There is a warning about the traders not being subscribed who own the orders at exchange. Normally this would cause an error but the command messenger.set_asserting(False) allows us to continue.

Henry's order is now at exchange:

In [62]:
exchange

                              tid       
otype                         Ask    Bid
price time qid       oid qty            
100   0    1.000000  1   1    NaN      0
101   0    3.000000  3   1    NaN      1
102   0    5.000000  5   1    NaN      2
103   0    7.000000  7   1    NaN      3
104   0    9.000000  9   1    NaN      4
105   0    11.000000 11  1    NaN      5
109   1    12.000003 50  2    NaN  Henry
110   0    6.000000  6   1      3    NaN
111   0    8.000000  8   1      4    NaN
112   0    10.000000 10  1      5    NaN

1. The Get Order message sent to Henry will cause it to send an order to the exchange (generated by the getorder method).
2. Since it is a bid above best ask, it will execute. 
3. Exchange will contact the counterparties with executions
4. Exchange will contact Henry with executions and trade ammendement since this is a partial fill
5. Henry will bookkeep and update internal records

In [63]:
messenger.publish_log(1)

,fromm,too,subject,time,order
0,a god,Henry,New Customer Order,1,"(Henry, Bid, 109, 5, 1, None, 50)"
1,a god,Henry,Get Order,1,None
2,Henry,exchange1,New Exchange Order,1,"(Henry, Bid, 109, 5, 1, None, 50)"
3,exchange1,Henry,Confirm,1,"(Henry, Bid, 109, 5, 1, 12, 50)"
4,exchange1,Henry,Fill,1,"(Henry, Bid, 107, 1, 1, 12.0, None, 1)"
5,exchange1,Henry,Ammend,1,"(Henry, Bid, 109, 4, 1, 12.000001, 50)"
6,exchange1,Henry,Fill,1,"(Henry, Bid, 108, 1, 1, 12.000001, None, 1)"
7,exchange1,Henry,Ammend,1,"(Henry, Bid, 109, 3, 1, 12.000002, 50)"
8,exchange1,Henry,Fill,1,"(Henry, Bid, 109, 1, 1, 12.000002, None, 1)"
9,exchange1,Henry,Ammend,1,"(Henry, Bid, 109, 2, 1, 12.000003, 50)"


In [64]:
henry.orders_dic

OrderedDict([(50,
              {'Original': Order(tid='Henry', otype='Bid', price=109, qty=5, time=1, qid=None, oid=50),
               'submitted_quotes': [Order(tid='Henry', otype='Bid', price=109, qty=5, time=1, qid=12, oid=50),
                Order(tid='Henry', otype='Bid', price=109, qty=4, time=1, qid=12.000001, oid=50),
                Order(tid='Henry', otype='Bid', price=109, qty=3, time=1, qid=12.000002, oid=50),
                Order(tid='Henry', otype='Bid', price=109, qty=2, time=1, qid=12.000003, oid=50)],
               'qty_remain': 2})])

The trade report details any executions

In [65]:
#get last order in orders_dic - this is an ordered dic
oid=next(reversed(henry.orders_dic))

pd.DataFrame(henry.blotter[oid])

,tid,otype,client_price,order_qty,order_issue_time,accession_time,qid,oid,exec_time,exec_qty,exec_price,profit,improvement,BS,status
0,Henry,Bid,109,5,1,1,12.000000,50,1,1,107,2,2,Buy,partial
1,Henry,Bid,109,4,1,1,12.000001,50,1,1,108,1,1,Buy,partial
2,Henry,Bid,109,3,1,1,12.000002,50,1,1,109,0,0,Buy,partial


Because there were 3 legs to the execution, there are 3 ammendments:

The fills and the ammendments have been reported to the trader.

Running profit is stored in the balance attribute and the executions are stored in the blotter, it is the difference between the original order price and the executed price.

In [66]:
assert henry.balance==pd.DataFrame(henry.blotter[oid]).profit.sum()
henry.balance

3

The original order was for 5, 3 units have been executed so there should be a remaining order of qty 2. Note that qid is incremented for the ammended order on exchange.

In [67]:
henry.orders_dic[50]['qty_remain']

2

Check that the exchange agrees:

In [68]:
exchange

                              tid       
otype                         Ask    Bid
price time qid       oid qty            
100   0    1.000000  1   1    NaN      0
101   0    3.000000  3   1    NaN      1
102   0    5.000000  5   1    NaN      2
103   0    7.000000  7   1    NaN      3
104   0    9.000000  9   1    NaN      4
105   0    11.000000 11  1    NaN      5
109   1    12.000003 50  2    NaN  Henry
110   0    6.000000  6   1      3    NaN
111   0    8.000000  8   1      4    NaN
112   0    10.000000 10  1      5    NaN

The order book will act how you would expect so if another Buy order is entered at 109, it will get a lower priority:

In [69]:
john=giveaway(tid='John',exchange=exchange,timer=timer,time=0,messenger=messenger)
new_order=Order(tid='John', otype='Bid', price=109, qty=1, qid=None,time=john.time, oid=51)
message=Message(too='John',fromm='a god',time=timer.time,order=new_order,subject='New Customer Order')
messenger.send(message)
message=Message(too='John',fromm='a god',time=timer.time,order=None,subject='Get Order')
messenger.send(message)

In [70]:
print(exchange)

                              tid       
otype                         Ask    Bid
price time qid       oid qty            
100   0    1.000000  1   1    NaN      0
101   0    3.000000  3   1    NaN      1
102   0    5.000000  5   1    NaN      2
103   0    7.000000  7   1    NaN      3
104   0    9.000000  9   1    NaN      4
105   0    11.000000 11  1    NaN      5
109   1    12.000003 50  2    NaN  Henry
           16.000000 51  1    NaN   John
110   0    6.000000  6   1      3    NaN
111   0    8.000000  8   1      4    NaN
112   0    10.000000 10  1      5    NaN


Check with a sell order of quantity 2 at 109 that Henry is completed not John.

In [71]:
timer.next_period()
paul=giveaway(tid='Paul',exchange=exchange,timer=timer,time=0,messenger=messenger)
new_order=Order(tid='Paul', otype='Ask', price=109, qty=2, qid=None,time=paul.time, oid=52)
message=Message(too='Paul',fromm='a god',time=timer.time,order=new_order,subject='New Customer Order')
messenger.send(message)
message=Message(too='Paul',fromm='a god',time=timer.time,order=None,subject='Get Order')
messenger.send(message)

Recipient not subscribed: SD
Recipient not subscribed: SD


In [72]:
print(exchange)

                        tid      
otype                   Ask   Bid
price time qid oid qty           
100   0    1   1   1    NaN     0
101   0    3   3   1    NaN     1
102   0    5   5   1    NaN     2
103   0    7   7   1    NaN     3
104   0    9   9   1    NaN     4
105   0    11  11  1    NaN     5
109   1    16  51  1    NaN  John
110   0    6   6   1      3   NaN
111   0    8   8   1      4   NaN
112   0    10  10  1      5   NaN


Since the trade is completed, the trader's orders_dic is wiped

In [73]:
henry.orders_dic

OrderedDict()

But the history of the order is record if trader.history=True

In [74]:
henry.orders_dic_hist

{50: {'Original': Order(tid='Henry', otype='Bid', price=109, qty=5, time=1, qid=None, oid=50),
  'submitted_quotes': [Order(tid='Henry', otype='Bid', price=109, qty=5, time=1, qid=12, oid=50),
   Order(tid='Henry', otype='Bid', price=109, qty=4, time=1, qid=12.000001, oid=50),
   Order(tid='Henry', otype='Bid', price=109, qty=3, time=1, qid=12.000002, oid=50),
   Order(tid='Henry', otype='Bid', price=109, qty=2, time=1, qid=12.000003, oid=50)],
  'qty_remain': 0,
  'completion_time': 2,
  'status': 'complete'}}

and executions are stored on the blotter

In [75]:
pd.DataFrame(henry.blotter[50])

,tid,otype,client_price,order_qty,order_issue_time,accession_time,qid,oid,exec_time,exec_qty,exec_price,profit,improvement,BS,status
0,Henry,Bid,109,5,1,1,12.000000,50,1,1,107,2,2,Buy,partial
1,Henry,Bid,109,4,1,1,12.000001,50,1,1,108,1,1,Buy,partial
2,Henry,Bid,109,3,1,1,12.000002,50,1,1,109,0,0,Buy,partial
3,Henry,Bid,109,2,1,1,12.000003,50,2,2,109,0,0,Buy,complete
